Fonction à importer


In [191]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Sklearn functionality
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn import preprocessing
# Keras functionality
import requests
# Ignore warnings
pd.options.mode.chained_assignment = None

Récupération des données

In [192]:
# Import data
stat_equipe_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/Statistiques-Ligue-1#stats_squads_standard_for").content)[0]
matchs_ligue1_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/calendrier/Scores-et-tableaux-Ligue-1#sched_2022-2023_13_1").content)[0]
matchs_ligue1_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/calendrier/Calendrier-et-resultats-2021-2022-Ligue-1#sched_all").content)[0]
matchs_ligue1_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/calendrier/Calendrier-et-resultats-2020-2021-Ligue-1#sched_all").content)[0]
matchs_ligue1_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/calendrier/Calendrier-et-resultats-2019-2020-Ligue-1#sched_2019-2020_13_1").content)[0]
matchs_ligue1_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/calendrier/Calendrier-et-resultats-2018-2019-Ligue-1#sched_all").content)[0]

Lier les données

In [193]:
list_concat = [matchs_ligue1_2019,matchs_ligue1_2020, matchs_ligue1_2021, matchs_ligue1_2022, matchs_ligue1_2023]
matchs_ligue1 = pd.concat(list_concat)

Nettoyer les données

In [194]:
# Enlever les colonnes inutiles
matchs_ligue1 = matchs_ligue1[matchs_ligue1["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_trier= matchs_ligue1.drop(columns=["Sem.","Jour","Tribune","Arbitre","Notes","Rapport de match","Affluence","xG.1", "xG","Heure"],axis=1)
# Enlever les lignes inutiles
matchs_trier = matchs_trier[matchs_trier["Date"].notna()]
# Mettre les dates au bon format
matchs_trier["Date"] = pd.to_datetime(matchs_trier["Date"])
# Garder les matchs joués
matchs_joue = matchs_trier.dropna()
# Remettre les index à 0
matchs_joue.reset_index(drop=True, inplace=True)
# Ajouter les colonnes but_domicile et but_exterieur
matchs_joue["but_domicile"] = matchs_joue["Score"].str[0]
matchs_joue["but_exterieur"] = matchs_joue["Score"].str[2]
# garder les matchs à jouer
matchs_a_jouer = matchs_trier[matchs_trier["Score"].isna()]
# copier les colonnes Domicile et Extérieur dans les parties à jouer pour les retrouver après 
matchs_a_jouer.loc[:,["Domicile2"]]=matchs_a_jouer["Domicile"]
matchs_a_jouer["Extérieur2"]=matchs_a_jouer["Extérieur"]
# Mettres les buts en type int
matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)
# Ajouter les colonnes nombre de but et écarts de but
matchs_joue["nombre_de_but"] = matchs_joue["but_domicile"] + matchs_joue["but_exterieur"]
matchs_joue["ecart_de_but"] = matchs_joue["but_domicile"] - matchs_joue["but_exterieur"]

# Ajouter les colonnes resultat_domicile et resultat_exterieur
    # matchs_joue["resultat_domicile"] = 0
    # matchs_joue["resultat_exterieur"] = 0
    # for i in range(len(matchs_joue)):
    #     if matchs_joue["but_domicile"][i] > matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 2
    #         matchs_joue["resultat_exterieur"][i] = 0
    #     elif matchs_joue["but_domicile"][i] < matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 0
    #         matchs_joue["resultat_exterieur"][i] = 2
    #     else:
    #         matchs_joue["resultat_domicile"][i] = 1
    #         matchs_joue["resultat_exterieur"][i] = 1
# Diviser la date en jour, mois et année
matchs_joue["jour"] = matchs_joue["Date"].dt.day
matchs_joue["mois"] = matchs_joue["Date"].dt.month
matchs_joue["annee"] = matchs_joue["Date"].dt.year
matchs_joue.drop(columns=["Date"],axis=1,inplace=True)
# Diviser la date en jour, mois et année pour les matchs à jouer
matchs_a_jouer["jour"] = matchs_a_jouer["Date"].dt.day
matchs_a_jouer["mois"] = matchs_a_jouer["Date"].dt.month
matchs_a_jouer["annee"] = matchs_a_jouer["Date"].dt.year

Créer les données de test et d'entrainement

In [195]:
X = matchs_joue[["Domicile","Extérieur","jour","mois","annee"]]
y = matchs_joue[["nombre_de_but","ecart_de_but"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Encoder les colonnes Domicile et Extérieur pour les mettres en entier


In [196]:
cols = ['Domicile', 'Extérieur']
enc = preprocessing.LabelEncoder()
for col in cols:
    X_train[col] = X_train[col].astype('str')
    X_test[col] = X_test[col].astype('str')
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])
    matchs_a_jouer[col] = enc.fit_transform(matchs_a_jouer[col])

Création du modèle décision tree pour le nombre de but

In [197]:
# Build a logistic regression model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["nombre_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["nombre_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["nombre_de_but"], predictions1_dt_test))
matchs_a_jouer["nombre_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- mois <= 6.50
|   |--- mois <= 3.50
|   |   |--- annee <= 2021.50
|   |   |   |--- Domicile <= 11.50
|   |   |   |   |--- Extérieur <= 3.50
|   |   |   |   |   |--- mois <= 1.50
|   |   |   |   |   |   |--- Domicile <= 0.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Domicile >  0.50
|   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |--- mois >  1.50
|   |   |   |   |   |   |--- jour <= 10.50
|   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |   |--- jour >  10.50
|   |   |   |   |   |   |   |--- jour <= 22.00
|   |   |   |   |   |   |   |   |--- Extérieur <= 1.50
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- Extérieur >  1.50
|   |   |   |   |   |   |   |   |   |--- Domicile <= 7.50
|   |   |   |   |   |   |   |   |   |   |--- class: 4
|   |   |   |   |   |   |   |   |   |--- Domicile >  7.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |  

Création du model decision tree pour l'écart de but

In [198]:
# Build a logistic regression model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["ecart_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["ecart_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["ecart_de_but"], predictions1_dt_test))
matchs_a_jouer["ecart_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- Domicile <= 7.50
|   |--- jour <= 30.50
|   |   |--- Extérieur <= 10.50
|   |   |   |--- Domicile <= 5.50
|   |   |   |   |--- annee <= 2021.50
|   |   |   |   |   |--- Extérieur <= 8.50
|   |   |   |   |   |   |--- jour <= 22.50
|   |   |   |   |   |   |   |--- jour <= 10.00
|   |   |   |   |   |   |   |   |--- jour <= 2.00
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- jour >  2.00
|   |   |   |   |   |   |   |   |   |--- jour <= 4.50
|   |   |   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |   |   |   |--- jour >  4.50
|   |   |   |   |   |   |   |   |   |   |--- mois <= 1.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |   |   |--- mois >  1.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 3
|   |   |   |   |   |   |   |--- jour >  10.00
|   |   |   |   |   |   |   |   |--- jour <= 15.50
|   |   |   |   |   |  

Prévision des résultats

In [199]:
matchs_a_jouer["Score"] = ((matchs_a_jouer["nombre_de_but"]+matchs_a_jouer["ecart_de_but"])/2).astype(str) + " - " + ((matchs_a_jouer["nombre_de_but"]-matchs_a_jouer["ecart_de_but"])/2).astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","nombre_de_but","ecart_de_but"]]
matchs_prevu.head(15)

,Date,Domicile2,Extérieur2,Score,nombre_de_but,ecart_de_but
310,2020-03-13,Lyon,Reims,0.5 - 2.5,3,-2
311,2020-03-14,Montpellier,Marseille,-0.5 - 2.5,2,-3
312,2020-03-14,Nantes,Nîmes,3.0 - 3.0,6,0
313,2020-03-14,Strasbourg,Dijon,2.0 - -1.0,1,3
314,2020-03-14,Toulouse,Metz,3.5 - 0.5,4,3
315,2020-03-14,Brest,Lille,1.0 - 3.0,4,-2
316,2020-03-14,Amiens,Angers,1.5 - 2.5,4,-1
317,2020-03-15,Bordeaux,Rennes,-1.0 - 4.0,3,-5
318,2020-03-15,Monaco,Saint-Étienne,1.0 - 1.0,2,0
319,2020-03-15,Paris S-G,Nice,0.5 - 0.5,1,0
